In [1]:
#importing libraries and visualisation tools

# Data Manipulation
import numpy as np
import pandas as pd
import seaborn as sns
import os

#Statistic descriptives
from scipy.stats import skew
from scipy.stats import kurtosis

#splitting the dataset
from sklearn.model_selection import train_test_split

#Changinging the data type of date
from datetime import datetime

#Data scalling 
from sklearn.preprocessing import MinMaxScaler

# Plotting graphs
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Machine learning libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

#Evaluation techniques 
from sklearn.metrics import accuracy_score
from sklearn import metrics

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#Apple dataset
apple = pd.read_csv(r"C:\Users\Mr Mathonsi MM\Documents\Honours/Project\Data\AAPL.csv")

In [3]:
# Predictor variables For the APPLE DATASET
apple['Open-Close']= apple.Open -apple.Close
apple['High-Low']  = apple.High - apple.Low
apple = apple.dropna()
X = apple[['Open-Close', 'High-Low']]


# Target variable For the APPLE DATASET
Y = np.where(apple['Adj Close'].shift(-1)>apple['Adj Close'],1,-1)

#Viewing the first five rows of the Predictor variables
X.head()

Open-Close  High-Low
0    0.268928  0.292857
1    0.276785  0.328571
2   -0.345000  0.515714
3    0.316071  0.386429
4   -0.217143  0.391072

In [4]:
Y = (Y - min(Y))/(max(Y)- min(Y))

In [5]:
# Splitting the dataset Into training and tests dataset for Apple
split_percentage = 0.2
split = int(split_percentage*len(apple))

X_train = X[:split]
Y_train = Y[:split]

X_test = X[split:]
Y_test = Y[split:]

In [6]:
# Instantiate KNN learning model(k=5)
knn = KNeighborsClassifier(n_neighbors=160)

# fit the model
knn.fit(X_train, Y_train)

# Accuracy Score
accuracy_train = accuracy_score(Y_train, knn.predict(X_train))*100.0
accuracy_test = accuracy_score(Y_test, knn.predict(X_test))*100.0

print ('Train_data Accuracy for Apple dataset: %.2f' %accuracy_train)
print ('Test_data Accuracy for Apple dataset: %.2f' %accuracy_test)


Train_data Accuracy for Apple dataset: 53.60
Test_data Accuracy for Apple dataset: 53.07


In [12]:
a = metrics.mean_absolute_error(Y_train, knn.predict(X_train))
print(a)

0.46403712296983757


In [ ]:
%matplotlib inline
plt.style.use('classic')
#plt.figure(figsize=(10,5))

# Predicted Signal
apple['Predicted_Signal'] = knn.predict(X)

# SPY Cumulative Returns
apple['SPY_returns'] = np.log(apple['Adj Close']/apple['Adj Close'].shift(1))
Cumulative_SPY_returns = apple[split:]['SPY_returns'].cumsum()*100

# Cumulative Strategy Returns 
apple['Startegy_returns'] = apple['SPY_returns']* apple['Predicted_Signal'].shift(1)
Cumulative_Strategy_returns = apple[split:]['Startegy_returns'].cumsum()*100

# Plot the results to visualize the performance
#Plot for Apple dataset
#lt.subplot(2,2,1)
plt.figure(figsize=(16,5))
plt.plot(Cumulative_SPY_returns, color='r',label = 'SPY Cummulative Returns')
plt.plot(Cumulative_Strategy_returns, color='b', label = 'Strategy Returns')
plt.xlabel('')
plt.ylabel('Cummulative return')
plt.legend()
plt.title('Apple')
plt.savefig('knnapple.jpg')
plt.show()



In [ ]:
# Calculate Sharpe ratio
Std = Cumulative_Strategy_returns.std()
Sharpe = (Cumulative_Strategy_returns-Cumulative_SPY_returns)/Std
Sharpe = Sharpe.mean()
print('Sharpe ratio: %.2f'%Sharpe )

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Features construction 
apple['Open-Close'] = (apple.Open - apple.Close)/apple.Open
apple['High-Low'] = (apple.High - apple.Low)/apple.Low
apple['percent_change'] = apple['Adj Close'].pct_change()
apple['std_5'] =apple['percent_change'].rolling(5).std()
apple['ret_5'] = apple['percent_change'].rolling(5).mean()
apple.dropna(inplace=True)

# X is the input variable
X =apple[['Open-Close', 'High-Low', 'std_5', 'ret_5']]

# Y is the target or output variable
y = np.where(apple['Adj Close'].shift(-1) > apple['Adj Close'], 1, -1)

In [ ]:
# Total dataset length
dataset_length = apple.shape[0]

# Training dataset length
split = int(dataset_length * 0.2)
split

In [ ]:
# Splitiing the X and y into train and test datasets
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Print the size of the train and test dataset
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
clf = RandomForestClassifier(random_state=10)
# Create the model on train dataset
model = clf.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
print('Correct Prediction (%): ', accuracy_score(y_test, model.predict(X_test), normalize=True)*100.0)

In [ ]:
# Run the code to view the classification report metrics
from sklearn.metrics import classification_report
report = classification_report(y_test, model.predict(X_test))
print(report)

In [ ]:
# SPY Cumulative Returns
apple['SPY_returns'] = np.log(apple['Adj Close']/apple['Adj Close'].shift(1))
Cumulative_SPY_returns = apple[split:]['SPY_returns'].cumsum()


apple['strategy_returns'] = apple.percent_change.shift(-1) * model.predict(X)
#(apple.strategy_returns[split:]).cumprod().plot()
plt.plot(Cumulative_SPY_returns, color='r',label = 'Strategic Returns')
plt.ylabel('Returns (%)')
plt.title('Apple')
plt.savefig('rfapple1.jpg')
plt.show()
